In [1]:
# %env CUDA_VISIBLE_DEVICES=1

In [2]:
%env HF_ENDPOINT

'https://hf-mirror.com'

In [3]:
import accelerate

accelerator = accelerate.Accelerator(
    log_with="tensorboard", 
    project_dir=".xvla",
    fsdp_plugin=accelerate.FullyShardedDataParallelPlugin(),
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
from xvla_wlr.model import XVLA, XVLAProcessor, Trainer, get_peft_model, Action, Observation

model = XVLA.from_pretrained("2toINF/X-VLA-SoftFold")
processor = XVLAProcessor.from_pretrained("2toINF/X-VLA-SoftFold", use_fast=True)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.52G [00:00<?, ?B/s]

In [ ]:
model = get_peft_model(model)
model, processor = accelerator.prepare(model, processor)
trainer = Trainer(model, processor, accelerator=accelerator)

In [ ]:
import os

from datasets_wlr import WLRZhuangEpisodeDataset
from curobo.types.robot import RobotConfig
from xvla_wlr.model import DATA_DOMAIN_ID


dataset = WLRZhuangEpisodeDataset("samples/2026-01-21_demo_clothes/episode_0/data.json")
# TODO rm
# robot_config = RobotConfig.from_basic(
#     f"{os.getcwd()}/robots/piper_ros/src/piper_description/urdf/piper_description.urdf",
#     base_link="base_link",
#     ee_link="link8",
# )
domain_id = DATA_DOMAIN_ID["robomind-agilex"]


from xvla_wlr.dataset import XVLAWLRZhuangEpisodeDataset

# TODO
xvla_dataset = XVLAWLRZhuangEpisodeDataset(
    dataset=dataset,
    robot_config_left=RobotConfig.from_basic(
        f"{os.getcwd()}/robots/piper-dualarm/piper-dualarm.urdf",
        base_link="common_base_link",
        ee_link="left_link8",
    ),
    robot_config_right=RobotConfig.from_basic(
        f"{os.getcwd()}/robots/piper-dualarm/piper-dualarm.urdf",
        base_link="common_base_link",
        ee_link="right_link8",
    ),
    domain_id=domain_id,
    prefetch=True,
)

In [ ]:
from xvla_wlr.dataset import Action


dataset = xvla_dataset[:32]

while True:
    num_timesteps_per_iteration = 16
    num_timesteps_remaining = len(dataset)

    while True:
        observation = dataset[:num_timesteps_remaining]
        if len(observation) < num_timesteps_per_iteration:
            break

        action = Action.from_observation(
            observation,
            num_steps=num_timesteps_per_iteration,
        )

        action_next = action[1:]
        observation_current = observation[:len(action_next)]

        num_timesteps_remaining -= len(action)

        trainer.fit(
            observation=observation_current,
            action=action_next,
        )